<a href="https://colab.research.google.com/github/Zaicevvv/GoIT-DSandML-Private/blob/main/HW10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from surprise import accuracy, Dataset, Reader, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, train_test_split

Завантаження датасету

In [8]:
data=Dataset.load_builtin('ml-100k')

reader=Reader(rating_scale=(1,5))
data

Підбір параметрів для SVD, SVDpp та NMF за допомогою крос-валідації

In [11]:
algos = [SVD, SVDpp, NMF]

for algo in algos:
  cross_validate(algo(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9347  0.9416  0.9351  0.9358  0.9319  0.9358  0.0032  
MAE (testset)     0.7358  0.7410  0.7370  0.7378  0.7374  0.7378  0.0017  
Fit time          1.89    1.52    1.46    1.63    1.63    1.63    0.15    
Test time         0.30    0.12    0.25    0.12    0.25    0.21    0.08    
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9196  0.9090  0.9229  0.9155  0.9234  0.9181  0.0053  
MAE (testset)     0.7234  0.7139  0.7244  0.7195  0.7230  0.7208  0.0039  
Fit time          27.54   26.43   28.53   26.21   26.76   27.09   0.85    
Test time         5.37    4.91    4.67    5.13    4.60    4.93    0.29    
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (tests

Оцінимо середні RMSE та MAE кожної моделі. Бачимо, що SVDpp хоч і не сильно, але впоралася краще. Побудуємо модель.

In [14]:
trainset, testset = train_test_split(data, test_size=0.25)

predictions = SVDpp().fit(trainset).test(testset)
# Подивимось на результат
accuracy.rmse(predictions)

RMSE: 0.9260


0.9259576719408424

**Завдання ***  
Реалізуємо алгоритм колабораційної фільтрації з нуля. Для цього скористаємось домашньою роботою з 3-ого модуля. Якщо ми модифікуємо функцію втрат та розрахунок градієнтів, то зможемо побудувати алгоритм матричної факторизації.

Завантажимо дані.

In [30]:
from scipy.io import loadmat

movies = []
with open('movie_ids.txt',  encoding='ISO-8859-1') as f:
  movies = f.readlines()

movieNames = []
for movie in movies:
  parts = movie.split()
  movieNames.append(' '.join(parts[1:]).strip())

movieNames

['Toy Story (1995)',
 'GoldenEye (1995)',
 'Four Rooms (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Twelve Monkeys (1995)',
 'Babe (1995)',
 'Dead Man Walking (1995)',
 'Richard III (1995)',
 'Seven (Se7en) (1995)',
 'Usual Suspects, The (1995)',
 'Mighty Aphrodite (1995)',
 'Postino, Il (1994)',
 "Mr. Holland's Opus (1995)",
 'French Twist (Gazon maudit) (1995)',
 'From Dusk Till Dawn (1996)',
 'White Balloon, The (1995)',
 "Antonia's Line (1995)",
 'Angels and Insects (1995)',
 'Muppet Treasure Island (1996)',
 'Braveheart (1995)',
 'Taxi Driver (1976)',
 'Rumble in the Bronx (1995)',
 'Birdcage, The (1996)',
 'Brothers McMullen, The (1995)',
 'Bad Boys (1995)',
 'Apollo 13 (1995)',
 'Batman Forever (1995)',
 'Belle de jour (1967)',
 'Crimson Tide (1995)',
 'Crumb (1994)',
 'Desperado (1995)',
 'Doom Generation, The (1995)',
 'Free Willy 2: The Adventure Home (1995)',
 'Mad Love (1995)',
 'Nadja (1994)',
 'Net, The (1995

In [31]:
data = loadmat('movies.mat')
Y, R = data['Y'], data['R']

# Y is a 1682x943 matrix, containing ratings (1-5) of
# 1682 movies on 943 users

# R is a 1682x943 matrix, where R(i,j) = 1
# if and only if user j gave a rating to movie i

Y.shape, R.shape

((1682, 943), (1682, 943))

Ініціалізуємо випадкові фактори для користувачів і фільмів.

In [32]:
num_users = Y.shape[1]
num_movies = Y.shape[0]
num_factors = 10

user_factors = np.random.rand(num_users, num_factors)
movie_factors = np.random.rand(num_movies, num_factors)

Реалізуємо функцію втрат та розрахунок градієнтів для матричної факторизації.

In [46]:
def compute_loss(Y, R, user_factors, movie_factors, lambda_reg):
    prediction = movie_factors @ user_factors.T
    loss = np.sum(R * (Y - prediction) ** 2) + lambda_reg * (np.sum(user_factors**2) + np.sum(movie_factors**2))
    return loss

def gradient_descent_step(Y, R, user_factors, movie_factors, learning_rate, lambda_reg):
    prediction = movie_factors @ user_factors.T
    error = R * (Y - prediction)

    user_gradient = -2 * (error.T @ movie_factors) + 2 * lambda_reg * user_factors
    movie_gradient = -2 * (error @ user_factors) + 2 * lambda_reg * movie_factors

    user_factors -= learning_rate * user_gradient
    movie_factors -= learning_rate * movie_gradient

    return user_factors, movie_factors

Навчимо модель за допомогою градієнтного спуску.

In [47]:
def train(Y, R, user_factors, movie_factors, learning_rate, lambda_reg, iterations):
    for i in range(iterations):
        user_factors, movie_factors = gradient_descent_step(Y, R, user_factors, movie_factors, learning_rate, lambda_reg)
        if i % 100 == 0:
            loss = compute_loss(Y, R, user_factors, movie_factors, lambda_reg)
            print(f"Iteration {i}: Loss = {loss}")
    return user_factors, movie_factors

learning_rate = 0.001
lambda_reg = 0.1
iterations = 1000

user_factors, movie_factors = train(Y, R, user_factors, movie_factors, learning_rate, lambda_reg, iterations)

Iteration 0: Loss = 1859225.8476034966


<ipython-input-46-4e4f27f86c8f>:7: RuntimeWarning: overflow encountered in matmul
  prediction = movie_factors @ user_factors.T
<ipython-input-46-4e4f27f86c8f>:8: RuntimeWarning: invalid value encountered in multiply
  error = R * (Y - prediction)


Iteration 100: Loss = nan
Iteration 200: Loss = nan
Iteration 300: Loss = nan
Iteration 400: Loss = nan
Iteration 500: Loss = nan
Iteration 600: Loss = nan
Iteration 700: Loss = nan
Iteration 800: Loss = nan
Iteration 900: Loss = nan


Після навчання моделі ми можемо прогнозувати рейтинги.

In [48]:
predictions = user_factors @ movie_factors.T
predictions

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])